In [4]:
import scraping_class
logfile = 'log.txt'## name your log file.
connector = scraping_class.Connector(logfile)
import requests
from bs4 import BeautifulSoup

In [5]:
import pandas as pd
import numpy as np
import html5lib

In [131]:
def create_df(url):
    # Requesting URL with get
    response = requests.get(url) 
    
    # Getting course_id and year direct from URL
    course_id = url.split('/')[-1]
    year = url.split('/')[-3]

    # Converting to soup
    raw_html = response.text
    soup = BeautifulSoup(raw_html,'lxml')

    # Info used to scrabe certain tables (exam and on in the right corner)
    info = soup.find_all("dl", {'class':'dl-horizontal'})
    
    # Generating error list if course page is out of match
    error = []

    # Only selecting courses with 2 tabels i.e. exactly 1 exam and the info bar to the right
    # Then appending information of each side by 'dt' and 'dd' and pairing them
    if len(info) == 2:
        comp_info = pd.DataFrame()
        cleaned_id_text = ['Course Name']
        for i in info[0].find_all('dt'):
            cleaned_id_text.append(i.text)
        cleaned_id__attrb_text = [soup.find('h1',{'class':"courseTitle"}).text]
        for i in info[0].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)
        
        # If there is a blank space the lists can't be paired. Therfore if lengths dosen't match, list will be created by siblings if sibling isen't on the initial side.
        if len(info[1].find_all('dt')) == len(info[1].find_all('dd')):
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dd'):
                cleaned_id__attrb_text.append(i.text)
        else:
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dt'):
                if i.next_sibling in info[1].find_all('dt'):
                    cleaned_id__attrb_text.append('')
                else:
                    cleaned_id__attrb_text.append(i.next_sibling.text)

        # Appending year from URL
        cleaned_id_text.append('Year')
        cleaned_id__attrb_text.append(year)
        
        # Appending course id form URL
        cleaned_id_text.append('Course id')
        cleaned_id__attrb_text.append(course_id)
        
        # Scrabing data from workload table
        work = soup.find('ul',{'class':'list-unstyled workload clearfix'})
        workfi = work.find_all('li')
        
        # Geting 1 list of element, therfore has to be paire 1-3 2-4 etc.
        for i in range(len(workfi)):
            if i % 2 ==0:
                cleaned_id_text.append(workfi[i].text)
            else: 
                cleaned_id__attrb_text.append(workfi[i].text)
        
        # Appending content
        cleaned_id_text.append(soup.find_all('div',{'class':"course-content"})[0].find('a').text)
        cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-content"})[0].find('div').text)
        
        # Appending learning outcome
        cleaned_id_text.append(soup.find_all('div',{'class':"course-description"})[0].find('a').text)
        cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-description"})[0].find('div').text)

        comp_info['Id'] = cleaned_id_text
        comp_info['Attribute'] = cleaned_id__attrb_text
    else:
        error.append(url)
        comp_info = []
        
    return(comp_info, error)

In [132]:
comp_info,e = create_df('https://kurser.ku.dk/archive/2017-2018/course/A%C3%98KA08084U')
comp_info

,Id,Attribute
0,Course Name,AØKA08084U Advanced\r\nMicroeconometrics
1,Credit,"7,5 ECTS"
2,Type of assessment,"Written examination, 3 hours under invigilatio..."
3,Exam registration requirements,None.
4,Aid,Without aids
5,Marking scale,7-point grading scale
6,Censorship form,External censorship
7,Exam period,for the autumn semester 2017:19 December 2017 ...
8,Re-exam,for the autumn semester 2017:21 February 2018 ...
9,Language,English


In [130]:
url = 'https://kurser.ku.dk/archive/2017-2018/course/A%C3%98KA08084U'
response = requests.get(url)
    
course_id = url.split('/')[-1]
year = url.split('/')[-3]

raw_html = response.text
soup = BeautifulSoup(raw_html,'lxml')

info = soup.find_all("dl", {'class':'dl-horizontal'})



soup.find_all('div',{'class':"course-description"})[0].find('a').text
soup.find_all('div',{'class':"course-description"})[0].find('div').text

'The course will provide the student with a statistical toolbox\r\nthat can be used for the estimation of a wide range of\r\nmicroeconometric models. After completing the course, the student\r\nwill be able to:Knowledge:Understand how the estimated parameters rely on specific\r\nidentifying assumptions.Understand the principle of M-estimation in terms of estimation\r\nand inference, as well as key examples of M-estimators.Get a grasp of how the most common numerical optimizers\r\nwork.Get an overview and understand the fundamental features of the\r\navailable estimators depending on the nature of the data (e.g.,\r\nbinary, multinomial, censored, sample selection problem).Understand the fundamental differences between the classical\r\napproach and the Bayesian approach, and the advantages and\r\ndrawbacks of each approach for a given estimation\r\nproblem.\xa0Skills:Be able to discuss the identifying assumptions and use\r\nregressions or descriptive data analysis to assess the\r\nassump